In [1]:
import pandas as pd
import numpy as np

In [2]:
client = "kfc"
df = pd.read_excel(f"./{client}_results.xlsx")
df

,hostname,Impressions,Inviews,Fixations,Fixation/Inview Ratio,Average Fixation Duration,Total Fixation Duration,Client Total Fixation Duration,Sample Size
0,aftonbladet.se,1331295,1011735,610177,60.31,4000.27,440030.0,2.440873e+09,high
1,bloggar.aftonbladet.se,152367,108874,80450,73.89,3929.02,777946.0,3.160897e+08,low
2,klart.se,1479,789,380,48.16,1901.77,764510.0,7.226726e+05,medium
3,malservice.aftonbladet.se,8,7,2,28.57,653.58,8496.5,1.307160e+03,low
4,skonhetsredaktorerna.se,49216,38826,49216,126.76,0.00,0.0,0.000000e+00,low
5,svd.se,78543,51949,34806,67.00,8451.64,49332239.0,2.941678e+08,medium
6,tramsfrans.aftonbladet.se,49216,38826,0,0.00,NaN,0.0,NaN,low
7,tv.nu,4812,3408,1233,36.18,2272.64,518162.0,2.802165e+06,medium


In [3]:
df = df[df["Sample Size"].isin(["high", "medium"])]
df["group"] = 1

/tmp/ipykernel_12638/497368128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["group"] = 1


In [4]:
final_df = df.groupby(["group"]).apply(
    lambda x: pd.Series(
        {
            "Impressions": np.floor(np.average(x["Impressions"], weights=x["Impressions"])),
            "Inviews": np.floor(np.average(x["Inviews"], weights=x["Inviews"])),
            "Fixations": np.floor(np.average(x["Fixations"], weights=x["Fixations"])),
            "Fixation/Inview Ratio": round(np.average(x["Fixation/Inview Ratio"], weights=x["Impressions"]), 2),
            "Average Fixation Duration": np.floor(np.average(x["Average Fixation Duration"], weights=x["Fixations"])),
            "Total Fixation Duration": np.floor(np.average(x["Average Fixation Duration"], weights=x["Fixations"]) *  np.floor(np.average(x["Fixations"], weights=x["Fixations"])))
        }
    )
)
final_df = final_df.reset_index()
final_df

,group,Impressions,Inviews,Fixations,Fixation/Inview Ratio,Average Fixation Duration,Total Fixation Duration
0,1,1255917.0,961079.0,577685.0,60.59,4235.0,2.446703e+09


In [5]:
final_df = final_df[["Impressions", "Inviews", "Fixations", "Fixation/Inview Ratio", "Average Fixation Duration", "Total Fixation Duration"]]

final_df.to_excel(f"./{client}_results_consolidated.xlsx", index=False)